In [90]:
import os
import csv
import datetime
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

In [91]:
# 加载数据
def opencsv():
    # 使用 pandas 打开
    data = pd.read_csv('train.csv')
    data1 = pd.read_csv('test.csv')

    train_data = data.values[0:, 1:]  # 读入全部训练数据,  [行，列]
    train_label = data.values[0:, 0]  # 读取列表的第一列
    test_data = data1.values[0:, 0:]  # 测试全部测试个数据
    return train_data, train_label, test_data

In [92]:
# 加载数据
trainData, trainLabel, testData = opencsv()

In [93]:
def dRPCA(x_train, x_test, COMPONENT_NUM):
    print('=======降维处理=======')
    arrayTrainData = np.array(x_train)
    arrayTestData = np.array(x_test)
    pca = PCA(n_components=COMPONENT_NUM, whiten=False)
    pca.fit(trainData)
    pcaTrainData = pca.transform(arrayTrainData)
    pcaTestData = pca.transform(arrayTestData)

    print('特征数量:%s' % pca.n_components_)
    print('总方差占比:%s' % sum(pca.explained_variance_ratio_))
    return pcaTrainData, pcaTestData, pca.n_components_

In [94]:
# 特征工程
trainDataPCA, testDataPCA, components = dRPCA(trainData, testData, 0.8)

=======降维处理=======
特征数量:43
总方差占比:0.8003149736842471


In [95]:
# knn
def trainModelByKNN(data, label):
    clf = KNeighborsClassifier()  # default 5
    clf.fit(data, np.ravel(label))
    return clf

In [96]:
from sklearn.svm import SVC


# svm
def trainModelBySVM(data, label):
    clf = SVC(C=4, kernel='rbf')
    clf.fit(data, label)
    return clf

In [97]:
from sklearn.ensemble import RandomForestClassifier


# random forest
def trainModelByRF(data, label):
    clf = RandomForestClassifier(
        n_estimators=60,
        oob_score=True,
        max_depth=13, min_samples_split=120,
        min_samples_leaf=20, max_features=7,
        random_state=10,
    )
    clf.fit(data, label)
    return clf

In [98]:
from sklearn.neural_network import MLPClassifier


# MLP
def trainModelByCNN(data, label, components):
    clf = MLPClassifier(
        hidden_layer_sizes=(components,),
        activation='relu',
        alpha=0.0001,
        learning_rate='constant',
        learning_rate_init=0.001,
        max_iter=200,
        shuffle=True,
        random_state=10,
    )
    clf.fit(data, label)
    return clf

In [99]:
# knn模型训练
# clf = trainModelByKNN(trainDataPCA, trainLabel)
# svm模型训练
# clf = trainModelBySVM(trainDataPCA, trainLabel)
# rf模型训练
# clf = trainModelByRF(trainDataPCA, trainLabel)
# cnn模型训练
clf = trainModelByCNN(trainDataPCA, trainLabel, components)
# 结果预测
testLabel = clf.predict(testDataPCA)

In [100]:
# 导出结果
def saveResult(result, csvName):
    with open(csvName, 'w', newline='') as myFile:
        myWriter = csv.writer(myFile)
        firstRows = ['ImageId', 'Label']
        myWriter.writerow(firstRows)
        index = 0
        for i in result:
            tmp = []
            index = index + 1
            tmp.append(index)
            tmp.append(int(i))
            myWriter.writerow(tmp)

In [101]:
# knn结果导出
# saveResult(testLabel, 'result_knn.csv')
# svm结果导出
# saveResult(testLabel, 'result_svm.csv')
# rf结果导出
# saveResult(testLabel, 'result_rf.csv')
# cnn结果导出
saveResult(testLabel, 'result_cnn.csv')